## Preprocessing (`.CEL` file)

※ CEL files the 'raw' data files produced at the end of the array scan, and are normally deposited in gene expression databases like [GEO](http://www.ncbi.nlm.nih.gov/geo/). 

In [1]:
library(affy)
library(oligo)
library(pd.mogene.2.0.st)

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which, which.max, which.min


Loading required package: Biobase

Welcome to Bioconductor

    Vignettes contai

```R
# load the script from the internet that is used in install bioconductor
source("http://bioconductor.org/biocLite.R")

# Each of these commands tells Bioconductor to download and install each package
biocLite("affy")
biocLite("oligo")
biocLite("limma")

# Installing BiocManager
install.packages("BiocManager")
BiocManager::install("pd.mogene.2.0.st")
BiocManager::install("mogene20sttranscriptcluster.db")
# BiocManager::install()
```

### 1. Load `.CEL` file matrix and convert to Affymetrix Arrays

First, the **[Affy package in R](https://academic.oup.com/bioinformatics/article/20/3/307/185980)** was used to convert the `.CEL` file matrix, which is unified in the form of expression value.

In [2]:
basedir <- "../data/GSE62646_RAW"
setwd(basedir)

In [3]:
celFiles <- list.celfiles()
affyRaw  <- read.celfiles(celFiles)

Loading required package: pd.hugene.1.0.st.v1

Platform design info loaded.



Reading in : GSM1530765_Control_1.CEL
Reading in : GSM1530766_Control_2.CEL
Reading in : GSM1530767_Control_3.CEL
Reading in : GSM1530768_Control_4.CEL
Reading in : GSM1530769_Control_5.CEL
Reading in : GSM1530770_Control_6.CEL
Reading in : GSM1530771_Control_7.CEL
Reading in : GSM1530772_Control_8.CEL
Reading in : GSM1530773_Control_9.CEL
Reading in : GSM1530774_Control_10.CEL
Reading in : GSM1530775_Control_11.CEL
Reading in : GSM1530776_Control_12.CEL
Reading in : GSM1530777_Control_13.CEL
Reading in : GSM1530778_Control_14.CEL
Reading in : GSM1530779_Patient_1_admission.CEL
Reading in : GSM1530780_Patient_1_discharge.CEL
Reading in : GSM1530781_Patient_1_6_months.CEL
Reading in : GSM1530782_Patient_2_admission.CEL
Reading in : GSM1530783_Patient_2_discharge.CEL
Reading in : GSM1530784_Patient_2_6_months.CEL
Reading in : GSM1530785_Patient_3_admission.CEL
Reading in : GSM1530786_Patient_3_discharge.CEL
Reading in : GSM1530787_Patient_3_6_months.CEL
Reading in : GSM1530788_Patient_4_

In [4]:
affyRaw

GeneFeatureSet (storageMode: lockedEnvironment)
assayData: 1102500 features, 98 samples 
  element names: exprs 
protocolData
  rowNames: GSM1530765_Control_1.CEL GSM1530766_Control_2.CEL ...
    GSM1530862_Patient_28_6_months.CEL (98 total)
  varLabels: exprs dates
  varMetadata: labelDescription channel
phenoData
  rowNames: GSM1530765_Control_1.CEL GSM1530766_Control_2.CEL ...
    GSM1530862_Patient_28_6_months.CEL (98 total)
  varLabels: index
  varMetadata: labelDescription channel
featureData: none
experimentData: use 'experimentData(object)'
Annotation: pd.hugene.1.0.st.v1 

### 2. Normalizing Affy Data from CEL files

Next, **the Robust Multi-array Average (RMA) method** was performed to normalize the expression matrix.

In [5]:
eset <- rma(affyRaw)

Background correcting
Normalizing
Calculating Expression


In [6]:
eset

ExpressionSet (storageMode: lockedEnvironment)
assayData: 33297 features, 98 samples 
  element names: exprs 
protocolData
  rowNames: GSM1530765_Control_1.CEL GSM1530766_Control_2.CEL ...
    GSM1530862_Patient_28_6_months.CEL (98 total)
  varLabels: exprs dates
  varMetadata: labelDescription channel
phenoData
  rowNames: GSM1530765_Control_1.CEL GSM1530766_Control_2.CEL ...
    GSM1530862_Patient_28_6_months.CEL (98 total)
  varLabels: index
  varMetadata: labelDescription channel
featureData: none
experimentData: use 'experimentData(object)'
Annotation: pd.hugene.1.0.st.v1 

In [7]:
# Finally, save the data to an output file to be used by other programs, etc (Data will be log2 transformed and normalized)
write.exprs(eset,file="GSE62646.txt")

### 3. Adding Gene Annotation to Normalized Expression Output

Finally, the **Bioconductor** package in R was employed to convert the probe information into a gene symbol. When multiple probes corresponded to only one gene, the average between the probes were used for the final expression.

In [8]:
# Load annotation library
library(mogene20sttranscriptcluster.db)

Loading required package: AnnotationDbi

Loading required package: org.Mm.eg.db







In [9]:
# Strategy is to create data frame objects and merge them together - put expression info into a data frame
my_frame <- data.frame(exprs(eset))

In [10]:
# Put annotation information in a data frame.  To get specific fields, use packageNameSYMBOL, where the caps part names the type of data you're after
# To get a list of available annotation information, run the packagename with () at the end, i.e. mogene20sttranscriptcluster()
Annot <- data.frame(ACCNUM=sapply(contents(mogene20sttranscriptclusterACCNUM), paste, collapse=", "), SYMBOL=sapply(contents(mogene20sttranscriptclusterSYMBOL), paste, collapse=", "), DESC=sapply(contents(mogene20sttranscriptclusterGENENAME), paste, collapse=", "))

In [11]:
# Merge data frames together (like a database table join)
all <- merge(Annot, my_frame, by.x=0, by.y=0, all=T)

In [12]:
# Write out to a file:
write.table(all,file="GSE62646.ann.txt", sep="\t")

### Reference

- [Processing Affymetrix Gene Expression Arrays](http://homer.ucsd.edu/homer/basicTutorial/affymetrix.html)